In [ ]:
import time
from pathlib import Path
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options


url = 'https://contenido.bce.fin.ec/documentos/informacioneconomica/indicadores/real/PrecioPetroleoCrudoEcuatoriano.html'
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
data = []

driver = webdriver.Chrome(options=opts)
driver.get(url)
time.sleep(2)
table = driver.find_element(By.TAG_NAME, 'tbody')
rows = table.find_elements(By.TAG_NAME, 'tr')

data_dir = Path('data')
if (data_dir / 'oilPrice.csv').exists():
    df_prices = pd.read_csv(data_dir / 'oilPrice.csv')
    last_saved_period = df_prices.iloc[0]
    last_period = rows[0].text.split()
    if last_saved_period[0] == last_period[0] and last_saved_period[1] == last_period[1]:
        print('There is no new prices')
    else:
        for row in rows:
            data.append(row.text.split())
        df = pd.DataFrame(data,columns=['Year','Month','Price (USD)'])
        df['Price (USD)'] = df['Price (USD)'].str.replace(',','.').astype(float)
        df.to_csv(data_dir / 'oilPrice.csv')

for row in rows:
    data.append(row.text.split())
df = pd.DataFrame(data,columns=['Year','Month','Price (USD)'])
df['Price (USD)'] = df['Price (USD)'].str.replace(',','.').astype(float)
df.to_csv(data_dir / 'oilPrice.csv')

driver.quit()


In [46]:
import os
os.chdir('..')

In [73]:
import time
from pathlib import Path
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

def scrape_table_rows(driver):
    table = driver.find_element(By.TAG_NAME, 'tbody')
    rows = table.find_elements(By.TAG_NAME, 'tr')
    data = [row.text.split() for row in rows]
    
    df = pd.DataFrame(data,columns=['Year','Month','Price (USD)'])
    df['Price (USD)'] = df['Price (USD)'].str.replace(',','.').astype(float)
    df['Year'] = df['Year'].astype(int)

    return df

def update_csv(file_path, new_df):
    if file_path.exists():
        df_old = pd.read_csv(file_path)
        last_saved_year, last_saved_month = df_old.iloc[0,:2]
        last_new_year, last_new_month = new_df.iloc[0,:2]

        if (last_saved_year == last_new_year) and (last_saved_month == last_new_month):
            print('There are no new prices.')
            return
        
    
    new_df.to_csv(file_path,index=False)
    print('Data Updated')

if __name__ == '__main__':
    url = 'https://contenido.bce.fin.ec/documentos/informacioneconomica/indicadores/real/PrecioPetroleoCrudoEcuatoriano.html'
    data_dir = Path('data')
    data_dir.mkdir(exist_ok=True)
    file_path = data_dir / 'oilPrice.csv'
    opts = Options()
    opts.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    driver = webdriver.Chrome(options=opts)
    driver.get(url)
    time.sleep(2)

    df = scrape_table_rows(driver)
    update_csv(file_path, df)

    driver.quit()


There are no new prices.
